# Fitle Overview

`fitle` is a lightweight Python framework for building statistical models, compiling them into efficient machine code with [Numba](https://numba.pydata.org/), and fitting to data using [iMinuit](https://iminuit.readthedocs.io/).

## Typical Workflow

1. **Define a Model** by doing symbolic arithmetic on components and fitting parameters
2. **Pipe the model** into a cost function such as Negative Log Likelihood (NLL) or χ², along with the data to be modeled
3. **Minimize** the cost function using the `fit()` method, which wraps iMinuit

## Key Concepts

The most important structure in the library is a **Model** that represents a mathematical expression which relates inputs (`INPUT`), constants, and parameters (`Param`s).

The user rarely needs to define a `Model` explicitly. Performing any arithmetic operation or function on a `Param` will return a `Model` representing that expression.

In [1]:
from fitle import Param, INPUT, gaussian, fit, Cost
import numpy as np

## API Reference

### Parameters

```python
# === Shorthand syntax (auto-named from variable) ===
a = ~Param                          # Unbounded param, named 'a'
sigma = +Param                      # Positive (>0), named 'sigma'
neg = -Param                        # Negative (<0), named 'neg'
mu, sigma = ~Param, +Param          # Tuple unpacking works

# === Explicit creation ===
Param('name')                       # Named parameter
Param(5.0)                          # With starting value
Param(0, 10)                        # With bounds [0, 10]
Param('name')(0, 10)(5.0)           # Chained: name, bounds, start

# === Constrained builders ===
Param.positive('name')              # bounds (1e-6, inf), start=1
Param.negative('name')              # bounds (-inf, -1e-6), start=-1
Param.unit('name')                  # bounds [0,1], start=0.5

# === Special params ===
INPUT                               # Placeholder for input data (x)
INDEX                               # Default index placeholder
index(*args)                        # Create INDEX param with range(*args)
```

In [2]:
# Creating parameters with shorthand syntax (auto-named)
a = ~Param                           # Unbounded, named 'a'
b = +Param                           # Positive, named 'b'
c = Param.unit()                     # Unit interval [0,1], auto-named 'c'
d = Param(0, 10)                     # Bounded [0, 10], auto-named 'd'
e = Param(5.0)                       # Start=5, auto-named 'e'

print(f"a (unbounded): {a}")
print(f"b (positive): {b}")
print(f"c (unit): {c}")
print(f"d (bounded 0-10): {d}")
print(f"e (start=5): {e}")

a (unbounded): a=0
b (positive): b=1
c (unit): c=0.5
d (bounded 0-10): d=5
e (start=5): e=5


### Model Construction

```python
# === Model Construction ===
const(value)                        # Wrap constant as Model
indecise(arr, index=INDEX)          # Select element at index position
identity(val)                       # Wrap in identity function
gaussian(mu=None, sigma=None)       # Gaussian PDF
exponential(tau=None)               # Exponential PDF
```

In [3]:
# Arithmetic on Params creates Models
a, b = ~Param, ~Param                # Auto-named 'a' and 'b'

# This creates a Model representing y = a*x + b
linear = a * INPUT + b
print(f"Linear model: {linear}")

# Built-in PDFs with auto-named params
mu, sigma = ~Param, +Param
gauss = gaussian(mu, sigma)
print(f"\nGaussian PDF: {gauss}")

Linear model: a=0 * INPUT + b=0

Gaussian PDF: 1 / (sigma=1 * 2.5066282746310002) * exp(-0.5 * ((INPUT - mu=0) / sigma=1) ** 2)


### Model Properties and Methods

```python
# === Model Properties ===
Model.params                        # List of THETA parameters
Model.free                          # List of free params (INPUT, INDEX)
Model.components                    # Sub-models split at iden() boundaries
Model.compiled                      # True if numba-compiled version exists

# === Model Methods ===
Model.__call__(x=None)              # Evaluate model; x required if INPUT free
Model.__mod__(subs)                 # Substitute: model % {param: value}
Model.__getitem__(map)              # Index substitution: model[i]
Model.grad(wrt=None)                # Symbolic gradient w.r.t. params
Model.simplify()                    # Algebraic simplification
Model.shape()                       # Infer output shape
Model.freeze()                      # Replace params with current values
Model.copy()                        # Deep copy of model tree
Model.compile()                     # JIT compile; stores in cache
```

In [4]:
# Model properties
mu, sigma, scale = ~Param, +Param, +Param
model = scale * gaussian(mu, sigma)

print(f"Parameters: {model.params}")
print(f"Free variables: {model.free}")

# Evaluate the model
x = np.linspace(-3, 3, 100)
y = model(x)
print(f"\nEvaluated at 100 points, shape: {y.shape}")

Parameters: [scale=_Param(min=1e-06, start=1.0, max=inf, value=1.0), sigma=_Param(min=1e-06, start=1.0, max=inf, value=1.0), mu=_Param(min=-inf, start=0.0, max=inf, value=0.0)]
Free variables: [INPUT]

Evaluated at 100 points, shape: (100,)


### Cost Functions

```python
# === Cost Functions ===
Cost.MSE(x, y)                      # Mean squared error
Cost.NLL(data)                      # Unbinned negative log-likelihood
Cost.binnedNLL(data, bins, range)   # Binned NLL
Cost.chi2(data, bins, range)        # Chi-squared (binned)
Cost.chi2(x=centers, y=counts)      # Chi-squared from histogram
Cost.bin_widths                     # Bin widths (if binned method)
```

In [5]:
# Example: Create cost functions
x_data = np.array([1, 2, 3, 4, 5])
y_data = np.array([2.1, 4.0, 5.9, 8.1, 10.0])

# MSE cost
mse_cost = Cost.MSE(x_data, y_data)

# Pipe a model to the cost function
slope, intercept = +Param, ~Param
model = slope * INPUT + intercept

model_cost = model | mse_cost
print(f"Cost model: {model_cost}")
print(f"\nInitial cost value: {model_cost()}")

Cost model: sum(([ 2.1  4.   5.9  8.1 10. ] - (slope=1 * [1 2 3 4 5] + intercept=0)) ** 2)

Initial cost value: 55.43


### Fitting

```python
# === Fitting ===
fit(model, grad=True, ncall=9999999, options={})
                                    # Returns FitResult

FitResult.fval                      # Best objective value
FitResult.success                   # Convergence flag
FitResult.values                    # Dict of {name: fitted_value}
FitResult.errors                    # Dict of {name: error}
FitResult.predict                   # Frozen model scaled by bin_widths
FitResult.minimizer                 # Underlying iminuit.Minuit object
```

In [6]:
# Fit the model
result = fit(model_cost)
print(result)

print(f"\nFitted values: {result.values}")
print(f"Errors: {result.errors}")

<FitResult fval=0.027, success=True>
slope: 1.99 ± 0.32
intercept: 0.04933 ± 1


Fitted values: {'slope': 1.990339520971359, 'intercept': 0.04932819961320994}
Errors: {'slope': 0.315999141076734, 'intercept': 1.0487999473161917}


### Reduction

```python
# === Reduction ===
Reduction(model, index_param, op=operator.add)
                                    # Reduce model over index with operator
                                    # Example: sum over n bins
```

Reductions are used for operations that sum or combine over an index, such as discrete convolutions.